# This file is created to Change the baseline dataset into a dupedataset 

In [2]:
import os
import pandas as pd

Address_df = pd.read_csv('Data_source/Sample_35_train/train_data/Address.csv')  
ContactPoint_df = pd.read_csv('Data_source/Sample_35_train/train_data/ContactPoint.csv')
HealthcareOrganization_df = pd.read_csv('Data_source/Sample_35_train/train_data/HealthcareOrganization.csv')
HealthcarePersonnel_df = pd.read_csv('Data_source/Sample_35_train/train_data/HealthcarePersonnel.csv')
Person_df = pd.read_csv('Data_source/Sample_35_train/train_data/Person.csv')
ServiceDepartment_df = pd.read_csv('Data_source/Sample_35_train/train_data/ServiceDepartment.csv')

# Specify the encoding to handle the decoding error
golden_record = pd.read_csv('train_golden_standard_duplicates.csv', encoding='latin1')

In [2]:
golden_record

,original_id,duplicate_id,entity_type,variation_type,field_name,original_value,varied_value
0,4e48ce3c-38a1-408d-b06f-0d4ac4092209,6b46dba4-5830-5ee3-ad90-707c1b1d29a6,Address,country_expansion,country,NL,Netherlands
1,9cdeb3e6-0870-415c-afcd-81b5d24bace4,389078b5-7572-5a7f-8f19-108e0ac2ede0,Address,postal_format,postalCode,64161,64161
2,68cce369-339b-4ce8-8bb3-66b225a3fcce,ccc37f61-f924-5456-99f9-041bbba8201c,Address,country_expansion,country,AT,Austria
3,d4e3fd71-e861-43fd-bca1-248409aaae6d,1eac3582-b4d2-5d52-b330-b354033b3d21,Address,country_expansion,country,AT,Austria
4,3464a4d6-7f6e-45a6-80b3-5a929e0394e7,dbc8a716-7908-50f6-b7ce-c93773364e1f,Address,city_typo,city,Zeijerveen,Zeijerven
...,...,...,...,...,...,...,...
2223,b6c843e4-1325-4452-ac29-c1bfb0791b94,b4f959ff-b472-5077-804b-fda26f7438d2,ContactPoint,email_typo,email,Carter.Cardiovascular@dept.healthcare.org,Cartre.Cardiovascular@dept.healthcare.org
2224,9e996246-f4d3-435c-81d7-4a3ce30481a5,eec45c8b-babe-53b0-9bdb-a033fd685021,ContactPoint,email_typo,email,ValenciaThomas.Infectious@dept.healthcare.org,ValenciaThomasg.Infectious@dept.healthcare.org
2225,7e48a1c8-ef4d-478a-b195-c85b1a78dfc0,0a2e9c6c-3e5c-53ed-b311-e0a18913cd01,ContactPoint,email_domain_change,email,Lam.Infectious@dept.healthcare.org,Lam.Infectious@dept.healthcare.nl
2226,2360db82-1a14-4f91-8a34-065b2b058a0e,0330be73-3337-54b1-8340-f3b87935bf53,ContactPoint,email_domain_change,email,Nash.Pharmacy@dept.healthcare.org,Nash.Pharmacy@dept.healthcare.nl


In [4]:
# Dictionary to store all dataframes by entity type
entity_dataframes = {}

# Get a static list of variable names first to avoid the iteration error
variable_names = list(locals().keys())

# Populate the dictionary with your existing dataframes
for var_name in variable_names:
    if var_name.endswith('_df'):
        entity_type = var_name.replace('_df', '')
        entity_dataframes[entity_type] = locals()[var_name]

# Iterate through each row in the golden record
for _, row in golden_record.iterrows():
    entity_type = row['entity_type']
    
    # Check if we have a dataframe for this entity type
    if entity_type in entity_dataframes:
        df = entity_dataframes[entity_type]
        
        # Find the matching row using original_id
        mask = df['identifier'] == row['original_id']
        
        
        # If a match is found, update the specified field with the varied value
        if mask.any():
            df.loc[mask, 'changed'] = True # Mark the row as changed
            field_name = row['field_name']
            if field_name in df.columns:
                df.loc[mask, field_name] = row['varied_value']
                df['identifier'] == row['duplicate_id']  # Ensure identifier is set correctly
                print(f"Updated {entity_type} - {row['original_id']} - {field_name}: {row['varied_value']}, identifier set to {row['duplicate_id']}")
            else:
                print(f"Warning: Field {field_name} not found in {entity_type} dataframe")
        else:
            print(f"Warning: No row with identifier {row['original_id']} found in {entity_type} dataframe")
    else:
        print(f"Warning: No dataframe found for entity type {entity_type}")

Updated Address - 70bef8fb-6598-42bf-8a18-bafe2df4a748 - country: Netherlands, identifier set to 3c16ef0a-4ed7-5f2b-98ed-7024e71532e9
Updated Address - ca5aa58d-2dd7-4003-9aa3-cdad289cd17d - city: Pnuthorst, identifier set to 304e011a-14fd-512e-a24a-30e09137ede4
Updated Address - 51cd2c07-a8f7-4b61-b9d2-b2de92bba701 - postalCode: 59270 , identifier set to cd5e2cad-578d-5a58-9332-405e82afde0f
Updated Address - e38690e7-e27a-48e9-91c3-d1bcc6be6432 - country: Netherlands, identifier set to 75883d2a-cd86-5150-afd8-15ff40163bfc
Updated Address - a54643b7-71b1-4f59-b268-495d0f2cd3b3 - city: Pietersbeirum, identifier set to b64f911f-d765-58fb-a0f7-aebab1da84c4
Updated Address - c3d2e249-a0b3-430f-8a23-12e869c458e7 - text: Levi-Schweiger-Weg 114B, identifier set to bdbc1033-86ae-5b61-89d4-cd668e701ed5
Updated Address - 15038b35-a28e-49e8-abb9-ac7bf62efe5e - postalCode: 7551DV, identifier set to 3d38bcca-1526-5ba0-a79b-c4ee12631d86
Updated Address - c0fef09a-cd63-497e-994d-ecd0a83baf04 - text: 

In [10]:
entity_dataframes['Address']

,identifier,text,city,postalCode,country,changed
0,c8a70639-eb11-47b3-a7a9-c3787c65c1e5,Lukassteeg 84,Oud Zuilen,5579 YN,Netherlands,True
1,d3290a4c-b5d3-4b16-a619-4cb1d71037d1,Anne Stravenue 82,Adreiside,45067,EE,True
2,9cdeb3e6-0870-415c-afcd-81b5d24bace4,Kaasik Field 107,Reetbury,64161,EE,NaN
3,971c702d-5bf4-4c04-ac64-2b4c49b25ded,Pimstraat 126,Lengel,3250 EH,NL,NaN
4,0cc36d8c-7786-4fe5-9675-ebf74fe30c9a,Cornelius-Mitterer-Ring 83,Sankt Valentin,3787,AT,True
...,...,...,...,...,...,...
316,a5acf1da-2eee-4c46-b558-1ebb398a0e87,Kadri Island 34,Jevgeifort,84956,EE,True
317,63cb5ee0-a457-4975-9455-4d76109bc754,Rand Point 116A,East Tiiuton,54724,EE,True
318,e9b33269-bd4c-47dc-9c8f-43e5c5b24dc7,Aleksandr Pines 41,Soouth Meelis,74495,EE,True
319,bd0cda09-0cb3-4bbf-af8c-b8a91754d025,Pukk Mission 3,Triinfurt,51891,Estonia,True


In [6]:
A_copy = Address_df.copy()
entity = A_copy[A_copy['identifier'] == '6b46dba4-5830-5ee3-ad90-707c1b1d29a6']
print(entity)

Empty DataFrame
Columns: [identifier, text, city, postalCode, country, changed]
Index: []


In [11]:
from faker import Faker

fake = Faker()

# Create a copy to avoid modifying original data
A_copy = entity_dataframes['Address'].copy()
CP_copy = entity_dataframes['ContactPoint'].copy()
P_copy =  entity_dataframes['Person'].copy()
HO_copy = entity_dataframes['HealthcareOrganization'].copy()
SD_copy = entity_dataframes['ServiceDepartment'].copy()
HP_copy = entity_dataframes['HealthcarePersonnel'].copy()

# Save old values
old_valuesHO = {
    'identifier': HO_copy['identifier'].copy(),
    'address': HO_copy['address'].copy(),
    'contactPoint': HO_copy['contactPoint'].copy()
}

# Generate new UUIDs for each field

HO_copy['identifier'] = HO_copy['identifier'].apply(lambda x: fake.uuid4())
HO_copy['address'] = HO_copy['address'].apply(lambda x: fake.uuid4())
HO_copy['contactPoint'] = HO_copy['contactPoint'].apply(lambda x: fake.uuid4())

# Create copies of dataframes
old_valuesSD = {
    'identifier': SD_copy['identifier'].copy(),
    'address': SD_copy['address'].copy(),
    'contactPoint': SD_copy['contactPoint'].copy()
}

# Generate new UUIDs for each field in ServiceDepartment_df
SD_copy['identifier'] = SD_copy['identifier'].apply(lambda x: fake.uuid4())
SD_copy['address'] = SD_copy['address'].apply(lambda x: fake.uuid4())
SD_copy['contactPoint'] = SD_copy['contactPoint'].apply(lambda x: fake.uuid4())

old_valuesHP = {
    'identifier': HP_copy['identifier'].copy()
}
# Generate new UUIDs for Healthcare Personnel identifiers
HP_copy['identifier'] = HP_copy['identifier'].apply(lambda x: fake.uuid4())

In [12]:
# Create a mapping of old to new UUIDs
HO_uuid_mapping = {
    old_valuesHO['identifier'].iloc[i]: HO_copy['identifier'].iloc[i]
    for i in range(len(HO_copy))
}

# Update references in other dataframes (using the copies)
# Update institution references in Healthcare_Personnel_df copy
HP_copy['institution'] = HP_copy['institution'].map(HO_uuid_mapping).fillna(HP_copy['institution'])

# Update isPartOf references in ServiceDepartment_df copy
SD_copy['isPartOf'] = SD_copy['isPartOf'].map(HO_uuid_mapping).fillna(SD_copy['isPartOf'])

# Update contact point references
ho_contact_mapping = {
    old_valuesHO['contactPoint'].iloc[i]: HO_copy['contactPoint'].iloc[i]
    for i in range(len(HO_copy))
}

# Update identifier in ContactPoint_df copy where it matches old contact points
CP_copy['identifier'] = CP_copy['identifier'].map(ho_contact_mapping).fillna(CP_copy['identifier'])

# Update address references
ho_address_mapping = {
    old_valuesHO['address'].iloc[i]: HO_copy['address'].iloc[i]
    for i in range(len(HO_copy))
}

# Update identifier in Address_df copy where it matches old addresses
A_copy['identifier'] = A_copy['identifier'].map(ho_address_mapping).fillna(A_copy['identifier'])

# Create mapping for ServiceDepartment
sd_uuid_mapping = {
    old_valuesSD['identifier'].iloc[i]: SD_copy['identifier'].iloc[i]
    for i in range(len(SD_copy))
}

# Create address mapping for ServiceDepartment
sd_address_mapping = {
    old_valuesSD['address'].iloc[i]: SD_copy['address'].iloc[i]
    for i in range(len(SD_copy))
}

# Create contact point mapping for ServiceDepartment
sd_contact_mapping = {
    old_valuesSD['contactPoint'].iloc[i]: SD_copy['contactPoint'].iloc[i]
    for i in range(len(SD_copy))
}

# Update references in Address_df copy for ServiceDepartment
A_copy['identifier'] = A_copy['identifier'].map(sd_address_mapping).fillna(A_copy['identifier'])

# Update references in ContactPoint_df copy for ServiceDepartment
CP_copy['identifier'] = CP_copy['identifier'].map(sd_contact_mapping).fillna(CP_copy['identifier'])

# Create mapping for Personnel
hp_uuid_mapping = {
    old_valuesHP['identifier'].iloc[i]: HP_copy['identifier'].iloc[i]
    for i in range(len(HP_copy))
}

# Update references in Person_df copy
P_copy['identifier'] = P_copy['identifier'].map(hp_uuid_mapping).fillna(P_copy['identifier'])

In [13]:
old_valuesHO

{'identifier': 0     71545a13-7a1d-4006-8d72-3104f77383c1
 1     6288e1a5-cc45-4821-98a6-416d1775336d
 2     5f3f5638-3870-4a14-b490-b6081dfc8352
 3     870f084c-7244-4536-a85e-25b4b3969057
 4     ccc14d51-73f6-40d8-a9f4-1cc04653a560
 5     fb5eb866-2640-411e-a9f2-c3c74505f4f6
 6     f2ad985f-ff3e-4ba1-8ac7-28b4a41865bf
 7     c2472fd6-03e9-4a02-8cea-2df00a66dc4e
 8     f164f9d8-4312-4ce2-9c21-51e17e56ac3d
 9     586f1721-0785-48d7-b118-2d235bf80676
 10    4ec985ff-94b2-4b9d-8881-9f421a42b629
 11    788c161e-f3cc-4d8a-8b16-78e45f20f406
 12    48622a67-4a29-4067-9c66-a27b6a325333
 13    862f78e1-25c2-4b6a-b575-bc6eeee84bcb
 14    f46860a2-a604-45af-b4db-1c1eaa66b464
 15    0b5aafef-85a1-482a-8761-585b4168705a
 16    3fee2fc7-2e2d-4fdf-b57b-8a920a12f3b3
 17    1e454241-45c1-40a8-a52b-a0ce627b585f
 18    839137a6-a977-4b71-9e6d-2d7e9f312acb
 19    f1c58f44-7e08-4c1b-8da6-a326451437d6
 20    909b471c-0d1f-4a7f-90fc-e92295be263b
 21    ce8c037e-e51c-4cdb-8247-34dfdd1f8e30
 22    53f3c158-42

In [14]:
import os

# Create the new directory if it doesn't exist
if not os.path.exists('testdata_dup'):
    os.makedirs('testdata_dup')

# Save each dataframe to CSV in the new directory
A_copy.to_csv('Data_source/Sample_35_train/train_data_dupe/Address.csv', index=False)
CP_copy.to_csv('Data_source/Sample_35_train/train_data_dupe/ContactPoint.csv', index=False)
HO_copy.to_csv('Data_source/Sample_35_train/train_data_dupe/HealthcareOrganization.csv', index=False)
HP_copy.to_csv('Data_source/Sample_35_train/train_data_dupe/HealthcarePersonnel.csv', index=False)
P_copy.to_csv('Data_source/Sample_35_train/train_data_dupe/Person.csv', index=False)
SD_copy.to_csv('Data_source/Sample_35_train/train_data_dupe/ServiceDepartment.csv', index=False)

In [15]:
import pandas as pd
import os

# Define paths
original_data_path = 'Data_source/Sample_35_train/train_data'
duplicate_data_path = 'Data_source/Sample_35_train/train_data_dupe'
golden_standard_path = 'train_golden_standard_duplicates.csv'
output_path = 'train_golden_standard_duplicates_final.csv'

# Read golden standard file
golden_df = pd.read_csv(golden_standard_path, encoding='latin1')

# Dictionary to store matching field values between original and duplicate datasets
field_value_to_id_mapping = {}

# Process each entity type
for _, row in golden_df.iterrows():
    entity_type = row['entity_type']
    field_name = row['field_name']
    varied_value = row['varied_value']
    
    # Define file paths based on entity type
    if entity_type == 'Address':
        original_file = 'Address.csv'
        duplicate_file = 'Address.csv'
    elif entity_type == 'ContactPoint':
        original_file = 'ContactPoint.csv'
        duplicate_file = 'ContactPoint.csv'
    elif entity_type == 'HealthcareOrganization':
        original_file = 'HealthcareOrganization.csv'
        duplicate_file = 'HealthcareOrganization.csv'
    elif entity_type == 'HealthcarePersonnel':
        original_file = 'HealthcarePersonnel.csv'
        duplicate_file = 'HealthcarePersonnel.csv'
    elif entity_type == 'Person':
        original_file = 'Person.csv'
        duplicate_file = 'Person.csv'
    elif entity_type == 'ServiceDepartment':
        original_file = 'ServiceDepartment.csv'
        duplicate_file = 'ServiceDepartment.csv'
    else:
        continue
    
    # Read duplicate dataset
    duplicate_df = pd.read_csv(os.path.join(duplicate_data_path, duplicate_file))
    
    # Look for records in the duplicate dataset that have the varied value
    # in the specified field
    if field_name in duplicate_df.columns:
        matching_rows = duplicate_df[duplicate_df[field_name] == varied_value]
        
        if not matching_rows.empty:
            # Get the identifier from the first matching row
            new_duplicate_id = matching_rows.iloc[0]['identifier']
            
            # Store the mapping for this specific row
            key = (entity_type, field_name, varied_value)
            field_value_to_id_mapping[key] = new_duplicate_id

# Create a new dataframe for the updated golden standard
updated_golden_df = golden_df.copy()

# Update the duplicate_id column based on our mappings
for i, row in updated_golden_df.iterrows():
    key = (row['entity_type'], row['field_name'], row['varied_value'])
    if key in field_value_to_id_mapping:
        updated_golden_df.at[i, 'duplicate_id'] = field_value_to_id_mapping[key]

# Save the updated golden standard file
updated_golden_df.to_csv(output_path, index=False, encoding='utf-8')

print(f"Updated golden standard file saved to {output_path}")
print(f"Total field-value mappings created: {len(field_value_to_id_mapping)}")
print(f"Duplicate IDs updated: {sum(updated_golden_df['duplicate_id'] != golden_df['duplicate_id'])}")

Updated golden standard file saved to train_golden_standard_duplicates_final.csv
Total field-value mappings created: 1913
Duplicate IDs updated: 2228
